In [52]:
from manim import *
import os
from klampt.math import so3
os.environ["PATH"] += ":/Library/TeX/texbin"
# from main import TwoLinkRobotArm

In [ ]:
class Gimbal3D(ThreeDScene):
    def construct(self):
        # Set camera view
        self.set_camera_orientation(phi=70 * DEGREES, theta=-45 * DEGREES)

        # Add to scene
        self._add_legend()
        self._display_coordinate_frame(np.array([-4, -5, -3]))
        display_commands = [[["\pi/2", "\pi/2", "0"], [PI/2, PI/2, 0]], [["0", "\pi/2", "0"], [0, PI/2, PI/2]]]

        for i in range(1):
            # Create the rings and arrow
            outer_ring, middle_ring, inner_ring, arrow = self._create_rings()
            self.add(outer_ring, middle_ring, inner_ring, arrow)  # Add the rings and arrow to the scene
            command = display_commands[i]
            command_text = self._display_commands(command[0])
            command_nums = command[1]
            
            ## Animations
            # Roll
            self.wait(0.5)
            self.play(
                command_text[0].animate.set_opacity(1),
                Rotate(outer_ring, angle=command_nums[0], axis=RIGHT, run_time=2),
                Rotate(middle_ring, angle=command_nums[0], axis=RIGHT, run_time=2), 
                Rotate(inner_ring, angle=command_nums[0], axis=RIGHT, run_time=2), 
                Rotate(arrow, angle=command_nums[0], axis=RIGHT, run_time=2),
            )
            
            # Pitch
            self.wait(0.5)
            # applied_rotation = so3.from_axis_angle(([1.0, 0, 0], command_nums[0]))
            # new_pitch_axis = np.array(so3.apply(applied_rotation, [0, 1, 0]))
            # print(new_pitch_axis)
            self.play(
                command_text[0].animate.set_opacity(0.4),
                command_text[1].animate.set_opacity(1),
                Rotate(middle_ring, angle=command_nums[1], axis=OUT, run_time=2), 
                Rotate(inner_ring, angle=command_nums[1], axis=OUT, run_time=2), 
                Rotate(arrow, angle=command_nums[1], axis=OUT, run_time=2),
            )

            # # Yaw
            # applied_rotation = so3.mul(so3.from_axis_angle((list(new_pitch_axis), command_nums[1])), applied_rotation)
            # new_yaw_axis = np.array(so3.apply(applied_rotation, [0, 0, 1]))
            # self.wait(0.5)
            # self.play(
            #     command_text[1].animate.set_opacity(0.4),
            #     command_text[2].animate.set_opacity(1),
            #     Rotate(inner_ring, angle=command_nums[2], axis=new_yaw_axis, run_time=2),
            #     Rotate(arrow, angle=command_nums[2], axis=new_yaw_axis, run_time=2)
            # )     

            # # Fade out the outer ring, middle ring, inner ring, arrow, and legend
            # self.play(
            #     FadeOut(outer_ring),
            #     FadeOut(middle_ring),
            #     FadeOut(inner_ring),
            #     FadeOut(arrow),
            #     *[FadeOut(item) for item in command_text],
            # )


    def _create_rings(self):
        # Parameters
        ring_thickness = 0.13  # tube radius

        # Outer Torus (Roll ring)
        outer_ring_sphere = Sphere(radius=0.2, checkerboard_colors=[RED, RED], stroke_color=RED)
        outer_ring_ob = Torus(major_radius=3, minor_radius=ring_thickness, checkerboard_colors=[RED, RED], stroke_color=RED)
        outer_ring_sphere.move_to(outer_ring_ob.get_center() + np.array([-3, 0, 0]))
        outer_ring = VGroup(outer_ring_ob, outer_ring_sphere)
        outer_ring.rotate(PI / 2, axis=UP)

        # Middle Torus (Pitch ring)
        middle_ring_sphere = Sphere(radius=0.2, checkerboard_colors=[GREEN, GREEN], stroke_color=GREEN)
        middle_ring_ob = Torus(major_radius=2.5, minor_radius=ring_thickness, checkerboard_colors=[GREEN, GREEN], stroke_color=GREEN)
        middle_ring_sphere.move_to(middle_ring_ob.get_center() + np.array([0, 2.5, 0]))
        middle_ring = VGroup(middle_ring_ob, middle_ring_sphere)
        middle_ring.rotate(PI / 2, axis=RIGHT)

        # Inner Torus (Yaw ring)
        inner_ring_sphere = Sphere(radius=0.2, checkerboard_colors=[BLUE, BLUE], stroke_color=BLUE)
        inner_ring_ob = Torus(major_radius=2.1, minor_radius=ring_thickness, checkerboard_colors=[BLUE, BLUE], stroke_color=BLUE)
        inner_ring_sphere.move_to(inner_ring_ob.get_center() + np.array([2.1, 0, 0]))
        inner_ring = VGroup(inner_ring_ob, inner_ring_sphere)

        # Position all rings at the center
        middle_ring.move_to(outer_ring.get_center())
        inner_ring.move_to(outer_ring.get_center())

        arrow = self._create_arrow_3D()
        arrow.move_to(outer_ring.get_center())

        return outer_ring, middle_ring, inner_ring, arrow
    
    def _display_coordinate_frame(self, origin=ORIGIN, axis_length=1.0, axis_thickness=0.05):
        """Create a 3D coordinate frame with X, Y, and Z axes."""
        x_axis = Arrow3D(
            start=origin,
            end=origin + np.array([axis_length, 0, 0]),
            resolution=8,
            thickness=axis_thickness,
            color=RED,
        )
        
        y_axis = Arrow3D(
            start=origin,
            end=origin + np.array([0, axis_length, 0]),
            resolution=8,
            thickness=axis_thickness,
            color=GREEN,
        )
        
        z_axis = Arrow3D(
            start=origin,
            end=origin + np.array([0, 0, axis_length]),
            resolution=8,
            thickness=axis_thickness,
            color=BLUE,
        )
        
        # Labels
        x_label = Text("X", color=RED).scale(0.5).next_to(x_axis.get_end(), RIGHT, buff=0.1)
        y_label = Text("Y", color=GREEN).scale(0.5).next_to(y_axis.get_end(), UP, buff=0.1)
        z_label = Text("Z", color=BLUE).scale(0.5).next_to(z_axis.get_end(), OUT, buff=0.1)
        
        # Group all elements
        frame = VGroup(x_axis, y_axis, z_axis, x_label, y_label, z_label)

        self.add(frame)

        # Make the text labels face the camera
        for label in [frame[3], frame[4], frame[5]]:  # The text labels
            self.add_fixed_orientation_mobjects(label)
        return frame
        
    def _add_legend(self):
        legend_items = VGroup(
            self._legend_entry(RED, "Roll (X-axis)"),
            self._legend_entry(GREEN, "Pitch (Y-axis)"),
            self._legend_entry(BLUE, "Yaw (Z-axis)")
        ).arrange(DOWN, aligned_edge=LEFT, buff=0.3).to_corner(UL)

        for item in legend_items:
            self.add_fixed_in_frame_mobjects(item)

    def _legend_entry(self, color, label_text):
        """Helper method to create a color dot and label pair"""
        color_box = Circle(radius=0.15, fill_color=color, fill_opacity=1, stroke_width=0)
        label = Text(label_text, font_size=24).next_to(color_box, RIGHT, buff=0.2)
        return VGroup(color_box, label)
    
    def _display_commands(self, commands):
        roll_command = MathTex(f"Roll: {commands[0]}", font_size=35, fill_opacity=0.5)
        pitch_command = MathTex(f"Pitch: {commands[1]}", font_size=35, fill_opacity=0.5)
        yaw_command = MathTex(f"Yaw: {commands[2]}", font_size=35, fill_opacity=0.5)

        command_group = VGroup(roll_command, pitch_command, yaw_command).arrange(DOWN, aligned_edge=LEFT, buff=0.3).to_corner(UR)

        for command in command_group:
            self.add_fixed_in_frame_mobjects(command)

        return command_group
    
    def _create_arrow_3D(self):
        vertex_coords = [
            [0.5, 1.5, 0],
            [0.5, -1.5, 0],
            [-0.5, -1.5, 0],
            [-0.5, 1.5, 0],
            [0, 0, 2]
        ]
        faces_list = [
            [0, 1, 4],
            [1, 2, 4],
            [2, 3, 4],
            [3, 0, 4],
            [0, 1, 2, 3]
        ]
        prism = Prism(dimensions=[3, 0.4, 0.2], fill_color=WHITE)
        pyramid = Polyhedron(vertex_coords, faces_list)
        # Prevent vertices from showing and set color of faces
        for i in range(5):
            pyramid.graph[i].scale(0)
            pyramid.faces[i].set_fill(color=WHITE, opacity=1).set_stroke(color=WHITE)  # Set the color of the faces to white
        pyramid.rotate(PI / 2, axis=UP)
        pyramid.scale(0.3)
        pyramid.move_to(prism.get_center() + np.array([1.5, 0, 0]))
        return VGroup(prism, pyramid)
    


In [54]:
%%manim -qm -v WARNING Gimbal3D

print("run")